In [3]:
!pip install plotly

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

def remove_first_0(number):
    return number[1:] if int(number[0]) == '0' else int(number)

xl = pd.read_excel(open('./xlsx/crimes-delit.xlsx', 'rb'), sheet_name='France_Entière')
effectifs_polices = pd.read_excel(open('./xlsx/effectifs_polices.xlsx', 'rb'))

cleaned_tab = effectifs_polices[effectifs_polices['MI/SG/DLPAJ/SDPA/BPA/2021'].astype(str).str.isdigit()]

In [9]:
# les 10 départements avec le plus de crimes sur les 20 derniers années
tab = []

xl = pd.ExcelFile('./xlsx/crimes-delit.xlsx')
for sheet_name in xl.sheet_names:
    try:
        int(sheet_name)
    except(ValueError):
        continue
    xl = pd.read_excel(open('./xlsx/crimes-delit.xlsx', 'rb'), sheet_name=sheet_name)
    xl = xl.drop(['Index', 'libellé index'], axis=1)
    df = xl.sum(axis = 0, numeric_only=True).reset_index(name=sheet_name).set_index('index').transpose()
    tab.append(df)
df = pd.concat(tab)
df.columns = pd.to_datetime(df.columns, format='_%Y_%m')

all_crimes = df.sum(axis = 1, numeric_only=True)
crimes_delits = all_crimes.sort_values(ascending=False).reset_index(name='crimes et délits').set_index('index')


In [21]:

crimes_delits_10 = crimes_delits.head(10)
fig = px.pie(crimes_delits_10, values='crimes et délits', names=crimes_delits_10.index, title='10 départements avec le plus de crimes')
fig.update_traces(textposition='inside', textinfo='value+label')
fig.show()

In [19]:
sum_effectifs_polices = cleaned_tab.groupby(['MI/SG/DLPAJ/SDPA/BPA/2021'])['Unnamed: 6'].sum().reset_index(name='counts')
sum_effectifs_polices.columns = ['numéro du département', 'Nombre d’agents de police municipale']
top_effectifs_polices = sum_effectifs_polices.sort_values(by='Nombre d’agents de police municipale', ascending=False).set_index('numéro du département')

fig = px.pie(top_effectifs_polices.head(10), values='Nombre d’agents de police municipale', names=top_effectifs_polices.head(10).index, title='10 départements avec le plus d\'effectif police municipal')
fig.update_traces(textposition='inside', textinfo='value+label')
fig.show()


In [22]:
nb_habitant = cleaned_tab.groupby(['MI/SG/DLPAJ/SDPA/BPA/2021'])['Unnamed: 4'].sum().reset_index(name='counts')
nb_habitant.columns = ['numéro du département', 'Nombre d\'habitant']
nb_habitant = nb_habitant.sort_values(by='Nombre d\'habitant', ascending=False).set_index('numéro du département')

fig = px.pie(nb_habitant.head(10), values='Nombre d\'habitant', names=nb_habitant.head(10).index, title='10 départements avec le plus d‘habitant')
fig.update_traces(textposition='inside', textinfo='value+label')
fig.show()
